In [1]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
#from spacy.lang.de import French
#parser = French()
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
import spacy
nlp=spacy.load("de_core_news_md")
import textacy
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer


In [4]:
X_full = pd.read_excel('../allData/Kopie-von-radarexport_june.xlsx',encoding="utf-8",
                      )
X_test_full = pd.read_excel('../allData/Kopie-von-radarexport_june.xlsx',encoding="utf-8")
"""
temporarily erasing columns: date and lien
"""
X_full=X_full.drop(["letzte relevante Änderung","Umsetzungsstand Norm","URL"
                    ,"Name der Quelle","Nr. "],axis=1)

y=X_full[["Rubrik"]]
X_full=X_full.drop(["Rubrik"],axis=1)
X_train, X_valid, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.5, test_size=0.5,
                                                                random_state=1)

In [5]:

spacy_stopwords = spacy.lang.de.stop_words.STOP_WORDS
stop=list(spacy_stopwords)
def spacy_tokenizer(sentence):
    sentence=textacy.preprocess_text(sentence,lowercase=True,no_punct=True,no_urls=True
                                    ,no_emails=True,no_currency_symbols=True)
    se=nlp(sentence)
    #mytokens = parser(sentence)
    mytokens = [word for word in se]
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word.strip().lower() for word in mytokens if word.strip().lower() not in stop ]
    return mytokens

In [6]:
tfvectorizer = TfidfVectorizer(tokenizer = spacy_tokenizer,ngram_range=(1,3))
classifier = SVC(C=150, gamma=2e-2, probability=True)
#from sklearn.ensemble import RandomForestClassifier
#classifier = sklearn.ensemble.RandomForestClassifier() 
preprocessor = ColumnTransformer(
    transformers=[
        
        ("article", tfvectorizer,"Normenbezeichnung"),
        
    ])
pipe=Pipeline([("preprocessor",preprocessor),("classifier",classifier)])

In [ ]:

pipe.fit(X_train,y_train)
pred=pipe.predict(X_valid)

/home/mo5/Desktop/desktop/python/jupyter1.0/jupyter1.0_env/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
accuracy_score(y_valid["Rubrik"],pred)

In [ ]:
output = pd.DataFrame({'Rubrik': pred} ,index=X_valid.index)
output.to_csv('allDataResultGER.csv')

In [ ]:
def accScore(sentence1,sentence2,res):
    dic={
    "article":sentence1,
    "title":sentence2
        
    }
    df=pd.DataFrame(dic,index=[0])
    predtest=pipe.predict(df)
    if( accuracy_score([res],predtest)==1):return "correct result"
    else : return "incorrect. Your pred is "+predtest[0]
accScore("ennahdha","rached","nahdha")

preds_test = pipe.predict(X_test_full.values.reshape(-1,))
output = pd.DataFrame({'id': X_test_full.index,
                       'label': preds_test})
output.to_csv('test_predictions.csv', index=False)